In [ ]:
import numpy as np
import scipy as sp
import pandas as pd
import warnings

import cython
%load_ext Cython
from iminuit import Minuit
idx = pd.IndexSlice

import clapy
import clasim


# Data for time series and histogramms

In [ ]:
asym_dist = clapy.dist()

dargs = {
        'samples': 10000,
        'nCells': 100,
        'mCells': 100,
        'GF': 0.95,
        'G1': 0.46,
        'S': 0.33,
        'G2M': 0.21,
        'sCells' : 0.3,
        'sSamples' : 0.2
}
dargs_d = dargs.copy()
dargs_d['sSamples'] = 1e-6
dargs_d['sCells'] = 1e-6

dTC = dargs['G1']+dargs['G2M']+dargs['S']
dFS = dargs['S']/dTC
X = np.arange(0,dargs['nCells']+1)
time_points = np.linspace(0.01,1.965625,22)
measure_times = np.ravel(np.array(time_points)[:,np.newaxis]*np.ones(dargs['samples']))


pdfs = list()
pdfm = list()
data_asy = list()
data_sym = list()

data_asy_d = list()
data_sym_d = list()

for t in time_points:
    #simulations asymetric
    data_asy.append(np.array(clasim.run(seed=int(np.random.rand()*1000),mode=1,times=[t],**dargs)) )
    data_asy_d.append(np.array(clasim.run(seed=int(np.random.rand()*1000),mode=1,times=[t],**dargs_d)) )
    #simulations symetric
    data_sym.append(np.array(clasim.run(seed=int(np.random.rand()*1000),mode=3,times=[t],**dargs)) )
    data_sym_d.append(np.array(clasim.run(seed=int(np.random.rand()*1000),mode=3,times=[t],**dargs_d)) )
    #pdfs
    pdfs.append( [asym_dist.pmf_f(dargs['nCells'],dTC,dFS,dargs['GF'],dargs['sCells'],dargs['sSamples'],t,i) for i in X] )
    #means
    pdfm.append( asym_dist.pmf_mean(dargs['nCells'],dTC,dFS,dargs['GF'],dargs['sCells'],dargs['sSamples'],t) )

    
with pd.HDFStore('paper_data.pandas',complevel=9) as st:
    st['data_asy'] = pd.Series(data_asy)
    st['data_sym'] = pd.Series(data_sym)
    
    st['data_asy_d'] = pd.Series(data_asy_d)
    st['data_sym_d'] = pd.Series(data_sym_d)
    
    st['pdfs']     = pd.Series(pdfs)
    st['pdfm']     = pd.Series(pdfm)


# Gernerate data for parameter recovery

In [ ]:
%%cython --force
# distutils: language = c++

#use --annotate if you wonder what kind of code it generates 
cimport cython
import numpy as np
cimport numpy as np #overwritten those from python with cython
from libc.math cimport exp, M_PI, sqrt, log
from iminuit.util import describe, make_func_code
from libcpp.map cimport map     
import scipy as sp



@cython.embedsignature(True)#dump the signatre so describe works
cpdef fitfunc(double t,double Tc,double r,double GF):
    cdef double res = 0
    if t<(Tc-Tc*r):
        res = GF/Tc*(t+Tc*r)
    else:
        res =  GF
    return res 
'''
@cython.embedsignature(True)#dump the signatre so describe works
cpdef double mypdf(double x, double mu, double std):
    #cpdef means generate both c function and python function
    cdef double norm = 1./(sqrt(2*M_PI*std))
    cdef double ret = exp(-1*(x-mu)*(x-mu)/(2.*std))*norm
    return ret
'''
@cython.embedsignature(True)#dump the signatre so describe works
cpdef double mypdfln(double x, double mu, double std):
    #cpdef means generate both c function and python function
    cdef double norm = (sqrt(2*M_PI*std*std))
    cdef double ret = (-1*(x-mu)*(x-mu)/(2.*std*std))-log(norm)
    return ret




cdef class Nowakowski_LH:
    cdef np.ndarray data
    cdef np.ndarray err
    cdef np.ndarray t
    cdef int ndata
    
    def __init__(self,data,t):
        self.data = data
        self.t = t
        self.ndata = len(data)
    
    @cython.embedsignature(True)#you need this to dump function signature in docstring
    def compute(self, double Tc,double r,double GF,double s):
        #this line is a cast not a copy. Let cython knows mydata will spit out double
        cdef np.ndarray[np.double_t, ndim=1] mydata = self.data
        cdef np.ndarray[np.double_t, ndim=1] myt = self.t
        cdef double loglh = 0.
        cdef double lmu = 0.
        cdef double ler = 0.
        for i in range(self.ndata):
            lmu = fitfunc(myt[i],Tc,r,GF)
            loglh -= mypdfln(mydata[i],lmu,s)
        return loglh



## with 5 samples

In [ ]:
default_args = {
        'samples': 5,
        'nCells': 100,
        'mCells': 100,
        'GF': 0.95,
        'G1': 0.46,
        'S': 0.33,
        'G2M': 0.21,
        'sCells' : 0.3,
        'sSamples' : 0.2 
}
warnings.filterwarnings('ignore')
calc_error = False
times=np.linspace(0.01,1.5,5)
default_args['times'] = times
#default_args['samples'] = int(np.round(120/(mm)))


index = pd.MultiIndex.from_product([[],[]],names=['std','index'])
# Priors for unknown model parameters

tmplist = []
tmplist2= []
for num in range(1000):
    Y,X = np.array(clasim.run(seed=2*num,mode=1,**default_args))
    

    lh = Nowakowski_LH(Y*1.0/default_args['nCells'],X)
    #lh.compute(0.4,5,3)

    mi_old = Minuit(lh.compute, Tc=1.0, r=0.3,GF=0.95,s=0.2,\
               error_s=0.1,error_r=0.1,error_GF=0.1,error_Tc=0.1,\
               limit_Tc=(0,2), limit_r=(0,1),limit_GF=(0,1),limit_s=(0,1),\
               errordef=0.5,print_level=0)
    mi_old.migrad(ncall=999999999);
    s = dict(mi_old.values)
    for ii in mi_old.errors:
        s.update({ii+"_error" : mi_old.errors[ii]})
    s.update({'model' : 'old'})

    if calc_error:
        try:
            tmp = mi_old.minos(sigma=2)                    
            for i in tmp:
                s.update({i+'_hpd46_l' :  tmp[i]['lower'] + mi_old.values[i] }) 
                s.update({i+'_hpd46_u' :  tmp[i]['upper'] + mi_old.values[i] }) 
        except:
            print('error in errorfind',s['model'])

    tmplist2.append(num)
    tmplist.append(s)



    Y,X = np.array(clasim.run(seed=2*num+1,mode=1,**default_args))

    lh = clapy.asym_lh(Y,X,100)
    mi_old = Minuit(lh.compute, Tc=1.0, r=0.3,GF=0.95,sigma_cell=0.3,sigma_sample=0.2,\
       error_sigma_cell=0.1,error_r=0.1,error_GF=0.1,error_Tc=0.1,error_sigma_sample=0.1,\
       limit_Tc=(0.00001,2), limit_r=(0.00001,1),limit_GF=(0,1),limit_sigma_cell=(0.00001,1),limit_sigma_sample=(0.00001,1),\
       errordef=0.5,print_level=0)
    mi_old.migrad();
    s = dict(mi_old.values)
    for ii in mi_old.errors:
        s.update({ii+"_error" : mi_old.errors[ii]})
    s.update({'model' : 'full'})

    if calc_error:
        try:
            tmp = mi_old.minos(sigma=2)                    
            for i in tmp:
                s.update({i+'_hpd46_l' :  tmp[i]['lower'] + mi_old.values[i] }) 
                s.update({i+'_hpd46_u' :  tmp[i]['upper'] + mi_old.values[i] }) 
        except:
            print('error in errorfind',s['model'])


    tmplist2.append(num)
    tmplist.append(s)   
    print('finiscged',num,s['model'],"t")

stat5 = pd.DataFrame(tmplist,index=tmplist2)
stat5.rename_axis({"GF":"GFf"}, axis="columns",inplace=True)
stat5.index.rename('N',inplace=True)
stat5['sSamples'] = 0.2
stat5['sCells'] = 0.3
stat5.set_index('sSamples', append=True, inplace=True)
stat5.set_index('sCells', append=True, inplace=True)
stat5.set_index('model', append=True, inplace=True)    

stat5 =  stat5.reorder_levels(['model','sSamples','sCells','N']) 

warnings.filterwarnings('default')

stat5 = stat5.sort_index()

with pd.HDFStore('paper_data.pandas',complib='zlib',complevel=9) as st:
    st['minuit_full_new_s5_para'] = default_args
    st['minuit_full_new_s5'] = stat5

## with 100 samples

In [ ]:
default_args = {
        'samples': 100,
        'nCells': 100,
        'mCells': 100,
        'GF': 0.95,
        'G1': 0.46,
        'S': 0.33,
        'G2M': 0.21,
        'sCells' : 0.3,
        'sSamples' : 0.2 
}
warnings.filterwarnings('ignore')
calc_error = False
times=np.linspace(0.01,1.5,5)
default_args['times'] = times

#default_args['samples'] = int(np.round(120/(mm)))


index = pd.MultiIndex.from_product([[],[]],names=['std','index'])
# Priors for unknown model parameters
SM = 456426
tmplist = []
tmplist2= []
for num in range(1000):
    Y,X = np.array(clasim.run(seed=num*2+SM,mode=1,**default_args))

    lh = Nowakowski_LH(Y*1.0/default_args['nCells'],X)
    #lh.compute(0.4,5,3)

    mi_old = Minuit(lh.compute, Tc=1.0, r=0.3,GF=0.95,s=0.2,\
               error_s=0.1,error_r=0.1,error_GF=0.1,error_Tc=0.1,\
               limit_Tc=(0,2), limit_r=(0,1),limit_GF=(0,1),limit_s=(0,1),\
               errordef=0.5,print_level=0)
    mi_old.migrad(ncall=999999999);
    s = dict(mi_old.values)
    for ii in mi_old.errors:
        s.update({ii+"_error" : mi_old.errors[ii]})
    s.update({'model' : 'old'})

    if calc_error:
        try:
            tmp = mi_old.minos(sigma=2)                    
            for i in tmp:
                s.update({i+'_hpd46_l' :  tmp[i]['lower'] + mi_old.values[i] }) 
                s.update({i+'_hpd46_u' :  tmp[i]['upper'] + mi_old.values[i] }) 
        except:
            print('error in errorfind',s['model'])

    tmplist2.append(num)
    tmplist.append(s)



    Y,X = np.array(clasim.run(seed=num*2+SM+1,mode=1,**default_args))


    lh = clapy.asym_lh(Y,X,100)
    mi_old = Minuit(lh.compute, Tc=1.0, r=0.3,GF=0.95,sigma_cell=0.3,sigma_sample=0.2,\
       error_sigma_cell=0.1,error_r=0.1,error_GF=0.1,error_Tc=0.1,error_sigma_sample=0.1,\
       limit_Tc=(0.00001,2), limit_r=(0.00001,1),limit_GF=(0,1),limit_sigma_cell=(0.00001,1),limit_sigma_sample=(0.00001,1),\
       errordef=0.5,print_level=0)
    mi_old.migrad();
    s = dict(mi_old.values)
    for ii in mi_old.errors:
        s.update({ii+"_error" : mi_old.errors[ii]})
    s.update({'model' : 'full'})

    if calc_error:
        try:
            tmp = mi_old.minos(sigma=2)                    
            for i in tmp:
                s.update({i+'_hpd46_l' :  tmp[i]['lower'] + mi_old.values[i] }) 
                s.update({i+'_hpd46_u' :  tmp[i]['upper'] + mi_old.values[i] }) 
        except:
            print('error in errorfind',s['model'])


    tmplist2.append(num)
    tmplist.append(s)   
    print('finiscged',num,s['model'],"t")

stat100 = pd.DataFrame(tmplist,index=tmplist2)
stat100.rename_axis({"GF":"GFf"}, axis="columns",inplace=True)
stat100.index.rename('N',inplace=True)
stat100['sSamples'] = 0.2
stat100['sCells'] = 0.3
stat100.set_index('sSamples', append=True, inplace=True)
stat100.set_index('sCells', append=True, inplace=True)
stat100.set_index('model', append=True, inplace=True)    

stat100 =  stat100.reorder_levels(['model','sSamples','sCells','N']) 

warnings.filterwarnings('default')

stat100 = stat100.sort_index()

with pd.HDFStore('paper_data.pandas',complib='zlib',complevel=9) as st:
    st['minuit_full_new_s100_para'] = default_args
    st['minuit_full_new_s100'] = stat100

## for different initial conditions

In [ ]:
default_args = {
        'samples': 100,
        'nCells': 100,
        'mCells': 100,
        'GF': 0.95,
        'G1': 0.46,
        'S': 0.33,
        'G2M': 0.21,
        'sCells' : 0.3,
        'sSamples' : 0.2 
}
warnings.filterwarnings('ignore')
calc_error = False
allim = []
sss = 0
SM = 928639
for start in np.linspace(0.01,0.2,19*1+1): #range(5,31,5):
    for leng in np.linspace(0.5,2,15*1+1):
        for n in range(10):
            times=np.linspace(start,start+leng,5)
            Y,X = np.array(clasim.run(seed=sss+SM,mode=1,times=times,**default_args))
            sss = sss+1
            lh = Nowakowski_LH(Y*1.0/default_args['nCells'],X)
            

            mi_old = Minuit(lh.compute, Tc=1.0, r=0.3,GF=0.95,s=0.2,\
                       error_s=0.1,error_r=0.1,error_GF=0.1,error_Tc=0.1,\
                       limit_Tc=(0,2), limit_r=(0,1),limit_GF=(0,1),limit_s=(0,1),\
                       errordef=0.5,print_level=0)
            mi_old.migrad(ncall=999999999);
            s = dict(mi_old.values)
            for ii in mi_old.errors:
                s.update({ii+"_error" : mi_old.errors[ii]})
            s.update({'N' : n})
            if calc_error:
                try:
                    tmp = mi_old.minos(sigma=2)                    
                    for i in tmp:
                        s.update({i+'_hpd46_l' :  tmp[i]['lower'] + mi_old.values[i] }) 
                        s.update({i+'_hpd46_u' :  tmp[i]['upper'] + mi_old.values[i] }) 
                except:
                    print('error in errorfind',s['model'])
            s.update({'leng' : leng})
            s.update({'start' : start})
            nowak = s.copy()



            Y,X = np.array(clasim.run(seed=sss+SM,mode=1,times=times,**default_args))
            sss = sss+1
            
            lh = clapy.asym_lh(Y,X,100)
            mi_old = Minuit(lh.compute, Tc=1.0, r=0.3,GF=0.95,sigma_cell=0.3,sigma_sample=0.2,\
               error_sigma_cell=0.1,error_r=0.1,error_GF=0.1,error_Tc=0.1,error_sigma_sample=0.1,\
               limit_Tc=(0.00001,2), limit_r=(0.00001,1),limit_GF=(0,1),limit_sigma_cell=(0.00001,1),limit_sigma_sample=(0.00001,1),\
               errordef=0.5,print_level=0)
            mi_old.migrad();
            s = dict(mi_old.values)
            for ii in mi_old.errors:
                s.update({ii+"_error" : mi_old.errors[ii]})
            s.update({'N' : n})
            s.update({'leng' : leng})
            s.update({'start' : start})

            if calc_error:
                try:
                    tmp = mi_old.minos(sigma=2)                    
                    for i in tmp:
                        s.update({i+'_hpd46_l' :  tmp[i]['lower'] + mi_old.values[i] }) 
                        s.update({i+'_hpd46_u' :  tmp[i]['upper'] + mi_old.values[i] }) 
                except:
                    print('error in errorfind',s['model'])


            print('finiscged')
            print(sss+SM,nowak,s)
            stat = pd.DataFrame([nowak,s],index=['old','full'])
            stat.set_index('start', append=True, inplace=True)
            stat.set_index('leng', append=True, inplace=True)
            stat.set_index('N', append=True, inplace=True)    

            allim.append( stat )
            #reorder_levels(['start','leng','model']) )

warnings.filterwarnings('default')

allimnew = pd.concat(allim).sort_index()

with pd.HDFStore('paper_data.pandas',complib='zlib',complevel=9) as st:
    st['s100_n10'] = allimnew
    st['s100_n10_para'] = default_args

In [ ]:
allimnew